确认设备以及cuda

In [19]:
import torch
from torch import cuda,optim 
print(torch.__version__);
print(torch.cuda.is_available())
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

2.6.0+cu126
True
cuda


0 - negative
1 - somewhat negative
2 - neutral
3 - somewhat positive
4 - positive

In [20]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader,random_split
df_train = pd.read_json('../weibo21_all.json', lines=True)
df_train.head()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3120\304779165.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_train = pd.read_json('../weibo21_all.json', lines=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3120\304779165.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_train = pd.read_json('../weibo21_all.json', lines=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3120\304779165.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' 

,id,label,category,images,content,timestamp
0,100000,0,0,"[{'path': 'image/100000_0.jpg'}, {'path': 'ima...",【三星折叠屏原型机曝光：双屏设计/非柔性屏】网友@黎启lee晒出三星的ProjcetV可折叠...,1970-01-18 16:46:49.446
1,100001,0,0,[{'path': 'image/100001_0.jpg'}],【能电鱼，更能电力找鱼：别以为电鳗只有把鱼电晕一招！】电鳗2米长的身体里有许多专门发电的细胞...,1970-01-17 18:11:57.263
2,100002,0,0,[{'path': 'image/100002_0.jpg'}],在iPad2发售几小时后，国外玩家就已经对它进行了拆解。想了解如此轻薄的机身如何容纳下这么多...,1970-01-16 01:04:58.932
3,100003,0,0,[{'path': 'image/100003_0.jpg'}],#乐享时光#现在科学家都这么闲了吗?谢谢你告诉我这么重要的结论啊！（转）,1970-01-17 13:05:04.203
4,100004,0,0,[{'path': 'image/100004_0.jpg'}],全国首家“无人银行”亮相大堂经理是智能机器人银行里没有忙碌的工作人员和拥挤的排队客户，而是各...,1970-01-18 15:12:18.840


提取标签以及输入

In [21]:
X = df_train['content']
Y = df_train['label'] 
print(df_train['label'].unique())

[0 1]


加载bert作为分词模型

In [22]:
from transformers import AutoTokenizer,BertModel
# 指定本地路径加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained("E:/mod/bert-base-uncased")

In [23]:
from torch import nn
import torch

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # 使用BERT分词器进行编码
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [24]:
# 最大句子长度
max_length = 128

# 创建 Dataset 实例
train_dataset = SentimentDataset(X, Y, tokenizer=tokenizer, max_length=max_length)

# 设置训练集和验证集的比例，比如 80% 训练集，20% 验证集
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

# 拆分数据集
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

# 创建 DataLoader
train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=64, shuffle=False)


# 查看一个 batch 的数据
for batch in train_loader:
    print(batch)
    break  # 只查看第一个 batch


{'input_ids': tensor([[ 101, 1740, 1840,  ...,    0,    0,    0],
        [ 101, 1001,  100,  ...,  100, 1740,  102],
        [ 101, 1001,  100,  ..., 1825, 1742,  102],
        ...,
        [ 101,  100, 1811,  ...,    0,    0,    0],
        [ 101,  100,  100,  ...,    0,    0,    0],
        [ 101, 1523,  100,  ...,  100, 1909,  102]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
        1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0])}


In [25]:
class BERTClassifier(nn.Module):
    def __init__(self, num_classes, dropout=0.7):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('E:/mod/bert-base-uncased')  # 加载预训练的 BERT 模型
        self.dropout = nn.Dropout(dropout)  # Dropout 层，防止过拟合
        self.linear = nn.Linear(self.bert.config.hidden_size, num_classes)  # 分类层

    def forward(self, input_ids, attention_mask):
        # 使用 BERT 获取隐藏状态（默认返回最后一层的所有隐藏状态）
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = bert_output.pooler_output  # 获取 [CLS] token 的表示
        dropout_output = self.dropout(pooled_output)  # 经过 Dropout
        logits = self.linear(dropout_output)  # 线性层输出分类结果
        return logits


In [26]:
# 实例化模型
num_classes = len(df_train['label'].unique())  # 类别数量
model = BERTClassifier(num_classes)  # 使用自定义的 BERT 分类模型
model.to(device)  # 将模型移动到设备上

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()  # 适用于分类任务
optimizer = optim.AdamW(model.parameters(), lr=2e-5,weight_decay=1e-2)

In [27]:
# 定义评估函数
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
# 定义评估函数
def evaluate(model, val_loader):
    model.eval()  # 设置模型为评估模式
    predictions, true_labels = [], []
    
    with torch.no_grad():  # 不需要计算梯度
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            # 前向传播
            outputs = model(input_ids, attention_mask)
            
            # 记录模型的预测值（取概率最大的类别）
            preds = F.softmax(outputs, dim=1).argmax(dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    
    # 计算准确率
    accuracy = accuracy_score(true_labels, predictions)
    return accuracy

In [ ]:
import matplotlib.pyplot as plt

# 初始化存储损失和准确率的列表
train_losses = []
val_accuracies = []

# Early stopping parameters
patience = 10
best_val_accuracy = 0
patience_counter = 0

num_epochs = 100


for epoch in range(num_epochs):
    model.train()  # 设置模型为训练模式
    total_loss = 0

    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # 前向传播
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)  # 记录训练损失

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}')

    # 每个 epoch 后进行评估
    val_accuracy = evaluate(model, val_loader)
    val_accuracies.append(val_accuracy)  # 记录验证准确率
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Accuracy: {val_accuracy:.4f}')

    # Early stopping logic
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        patience_counter = 0  # Reset patience counter
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            break
# 绘制损失和验证准确率图
plt.figure(figsize=(12, 5))

# 绘制损失图
plt.subplot(1, 2, 1)
plt.plot(range(1, len(train_losses) + 1), train_losses, marker='o', color='b', label='Training Loss')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.xticks(range(1, len(train_losses) + 1))  # 设置 x 轴刻度
plt.grid()
plt.legend()

# 绘制验证准确率图
plt.subplot(1, 2, 2)
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, marker='o', color='g', label='Validation Accuracy')
plt.title('Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.xticks(range(1, len(val_accuracies) + 1))  # 设置 x 轴刻度
plt.ylim(0, 1)  # 限定 y 轴范围
plt.grid()
plt.legend()

plt.tight_layout()  # 自动调整子图参数
plt.show()  # 显示图像


Epoch 1/100, Loss: 0.6506
Epoch 1/100, Validation Accuracy: 0.7119
Epoch 2/100, Loss: 0.5770
Epoch 2/100, Validation Accuracy: 0.7377
Epoch 3/100, Loss: 0.4985
Epoch 3/100, Validation Accuracy: 0.7645
Epoch 4/100, Loss: 0.4431
Epoch 4/100, Validation Accuracy: 0.7859
Epoch 5/100, Loss: 0.3741
Epoch 5/100, Validation Accuracy: 0.7908
Epoch 6/100, Loss: 0.3205
Epoch 6/100, Validation Accuracy: 0.7963
Epoch 7/100, Loss: 0.2807
Epoch 7/100, Validation Accuracy: 0.8248
Epoch 8/100, Loss: 0.2430
Epoch 8/100, Validation Accuracy: 0.8149
Epoch 9/100, Loss: 0.2040
Epoch 9/100, Validation Accuracy: 0.8363
Epoch 10/100, Loss: 0.1787
Epoch 10/100, Validation Accuracy: 0.8220
Epoch 11/100, Loss: 0.1468
Epoch 11/100, Validation Accuracy: 0.8401
Epoch 12/100, Loss: 0.1252
Epoch 12/100, Validation Accuracy: 0.8368
Epoch 13/100, Loss: 0.0982
Epoch 13/100, Validation Accuracy: 0.8324
Epoch 14/100, Loss: 0.0811
Epoch 14/100, Validation Accuracy: 0.8445
Epoch 15/100, Loss: 0.0776
Epoch 15/100, Validation 